# Testing the experiment classes


In [2]:
from common.experiment import KubemlExperiment, History, TrainOptions, TrainRequest
import pandas as pd
import glob
import numpy as np
import pickle

c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
# read the json
with open('./data/5.json', 'r') as f:
    content = f.read()


print(content)

In [ ]:
# read the content into a class

h = History.from_json(content)

exp = KubemlExperiment("test", None)
exp.history = h
exp.network_id = h.id

# exp.save('./tests/')

d = exp.to_dataframe()

In [ ]:
d.train_loss[0]

In [ ]:
import pandas as pd
e = pd.read_pickle('./tests/1a5c751e.pkl')
e

In [ ]:
import pandas as pd

h.to_dict()

new = {"id": h.id,
       **h.task.to_dict(),
       **h.task.options.to_dict()
       }

del new['options']

new

# # need to wrap the metrics in arrays
for k, v in h.data.to_dict().items():
    new[k] = [v]

new
#
import pandas as pd

pd.DataFrame(new)



### Test the experiment class

In [ ]:
request = TrainRequest(
    model_type='example',
    function_name='resnet',
    dataset='cifar10',
    lr=0.01,
    batch_size=32,
    epochs= 10,
    options= TrainOptions(
        default_parallelism=5,
        static_parallelism=True,
        validate_every=1,
        k = 10,
        goal_accuracy=90.0
    )
)

exp = KubemlExperiment(title='test experiment', request=request)
exp.network_id = '85878038'
exp.history = h

exp

In [ ]:
exp = KubemlExperiment('test', request= request)
exp.network_id = '85878038'

exp.check_if_task_finished()


## Merge experiments

Get the small dataframes of each experiment and merge them onto a single dataframe

In [4]:
import glob
from typing import List

files = glob.glob('./tests/resnet34/*.pkl')

dataframes: List[pd.DataFrame] = []




for f in files:
    _d = pd.read_pickle(f)
    dataframes.append(_d)

d = pd.concat(dataframes, ignore_index=True)

d

d.to_pickle('resnet_start.pkl')



In [5]:
pd.read_pickle('./resnet_start.pkl')


,id,model_type,batch_size,epochs,dataset,lr,function_name,default_parallelism,static_parallelism,validate_every,k,goal_accuracy,validation_loss,accuracy,train_loss,parallelism,epoch_duration
0,2b6b180b,example,128,1,cifar10,0.1,resnet,16,True,1,64,100,[4.6820562821400316],[10],[5.787266342118383],[16],[291.748263165]
1,4e8311db,example,128,1,cifar10,0.1,resnet,32,True,1,64,100,[4.120931245103667],[10],[6.655060767028957],[32],[235.414288248]
2,d649404e,example,128,1,cifar10,0.1,resnet,4,True,1,64,100,[2.317080847824676],[10],[3.632470758768785],[4],[738.871897622]
3,ef1139c9,example,128,1,cifar10,0.1,resnet,8,True,1,64,100,[3.4772339078444467],[10],[4.664617256242401],[8],[392.8319736]


## Download the datasets

In [ ]:
import torch
import torchvision.datasets as datasets

#### Mnist dataset

In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar10_tr = datasets.CIFAR10(root='./datasets/cifar10/', train=True, download=False, transform=None)
cifar10_test = datasets.CIFAR10(root='./datasets/cifar10/', train=False, download=False, transform=None)

In [ ]:
import keras
from keras.datasets import mnist, cifar10, cifar100
import numpy as np

file_names = ['x_train', 'y_train', 'x_test', 'y_test']


In [ ]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

# save the different train and test arrays in the datasets folder

for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/mnist/mnist_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)

### Cifar10 dataset

In [ ]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()


for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/cifar10/cifar10_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)


### Cifar100 dataset

In [ ]:
((trainX, trainY), (testX, testY)) = cifar100.load_data()


for name, data in zip(file_names, [trainX, trainY, testX, testY]):
    n = f'./datasets/cifar100/cifar100_{name}.npy'
    with open(n, 'wb') as f:
        print(f'Saving data of shape {data.shape}, to file {n}')
        np.save(n, data)

## Try to load the data in a custom made dataset

In [ ]:

class Cifar10Dataset(Dataset):

    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def __init__(self):
        # load the train data and labels
        self.data = np.load('datasets/cifar10/cifar10_x_train.npy')
        self.targets = np.load('datasets/cifar10/cifar10_y_train.npy')

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.transform(self.data[index]), self.targets[index]

In [ ]:
d = Cifar10Dataset()


In [ ]:
loader = DataLoader(d, batch_size=16)
i = iter(loader)


In [ ]:
a = next(i)

In [ ]:
x, y = a
x


In [1]:
#explore resnet

from torchvision.models.resnet import resnet18
from torchinfo import summary

r = resnet18(pretrained=True)

c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\diego\cs\thesis\venv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [32, 64, 16, 16]          9,408
├─BatchNorm2d: 1-2                       [32, 64, 16, 16]          128
├─ReLU: 1-3                              [32, 64, 16, 16]          --
├─MaxPool2d: 1-4                         [32, 64, 8, 8]            --
├─Sequential: 1-5                        [32, 64, 8, 8]            --
|    └─BasicBlock: 2-1                   [32, 64, 8, 8]            --
|    |    └─Conv2d: 3-1                  [32, 64, 8, 8]            36,864
|    |    └─BatchNorm2d: 3-2             [32, 64, 8, 8]            128
|    |    └─ReLU: 3-3                    [32, 64, 8, 8]            --
|    |    └─Conv2d: 3-4                  [32, 64, 8, 8]            36,864
|    |    └─BatchNorm2d: 3-5             [32, 64, 8, 8]            128
|    |    └─ReLU: 3-6                    [32, 64, 8, 8]            --
|    └─BasicBlock: 2-2                   [32, 64, 8, 8]            --
|

In [2]:

summary(r, input_size=(128, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [128, 64, 16, 16]         9,408
├─BatchNorm2d: 1-2                       [128, 64, 16, 16]         128
├─ReLU: 1-3                              [128, 64, 16, 16]         --
├─MaxPool2d: 1-4                         [128, 64, 8, 8]           --
├─Sequential: 1-5                        [128, 64, 8, 8]           --
|    └─BasicBlock: 2-1                   [128, 64, 8, 8]           --
|    |    └─Conv2d: 3-1                  [128, 64, 8, 8]           36,864
|    |    └─BatchNorm2d: 3-2             [128, 64, 8, 8]           128
|    |    └─ReLU: 3-3                    [128, 64, 8, 8]           --
|    |    └─Conv2d: 3-4                  [128, 64, 8, 8]           36,864
|    |    └─BatchNorm2d: 3-5             [128, 64, 8, 8]           128
|    |    └─ReLU: 3-6                    [128, 64, 8, 8]           --
|    └─BasicBlock: 2-2                   [128, 64, 8, 8]           --
|

In [ ]:
r

In [ ]:
from torchvision.models.vgg import vgg11

v = vgg11()
v

In [ ]:
from torch import nn

class LeNet(nn.Module):
    """ Definition of the LeNet network as per the 1998 paper

    Credits to https://github.com/ChawDoe/LeNet5-MNIST-PyTorch for the
    convenience of the network definition and the train loop found there
    """

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(256, 120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(120, 84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(84, 10)
        self.relu5 = nn.ReLU()

    def forward(self, x):
        y = self.conv1(x)
        y = self.relu1(y)
        y = self.pool1(y)
        y = self.conv2(y)
        y = self.relu2(y)
        y = self.pool2(y)
        y = y.view(y.shape[0], -1)
        y = self.fc1(y)
        y = self.relu3(y)
        y = self.fc2(y)
        y = self.relu4(y)
        y = self.fc3(y)
        y = self.relu5(y)
        return y

In [ ]:
n = LeNet()
summary(n, input_size=(32, 1, 28, 28))